# Sending event data directly into QuestDB

QuestDB is designed for high throughput, and can ingest streaming data at over 4 million events per second (using 12 CPUs and a fast drive). 

If your only reason to use Kafka in front of QuestDB is for ingestion speed, it might be the case you don't need it. 

Of course having Kafka as the ingestion layer gives you more flexibility, as you can easily send data to multiple consumers — other than QuestDB —, or you can restart the QuestDB server without stopping ingestion. On the other hand, adding Kafka means some (minor) extra latency and one more component to manage.

In the end, some teams would prefer to ingest into Kafka, and some directly into QuestDB. It all depends on your specific use case. QuestDB supports [multiple ways of ingesting data](https://questdb.io/docs/develop/insert-data/), but the fastest is by using the ILP protocol via the official clients.

This notebook uses the Python client for convenience, but the usage would be very similar using the [client libraries](https://questdb.io/docs/reference/clients/overview/) in NodeJs, Java, .Net, C/C++, Rust, or Go.

The notebook connects to port `9000` of the `questdb` container and sends data continuously (the client library will automatically flush every 75K rows by default). You can configure a delay between rows if you want by changing the the `BATCH` and `DELAY` constants.

The script ingest a mix of different types so you can experiment with different queries and dashboards. In a modern laptop you should be able to set a `BATCH` of 10000 and a `DELAY` of 0.1 seconds and still be responsive. Take into account that this will write potentially a lot of data in the `./questdb/questdb_root` folder at your repository root. It is safe to remove the contents (but please keep the folder) if you want to free disk space.

A live dashboard displaying this dataset can be seen at [http://localhost:3000/d/qdb-iot-demo/device-data-questdb-demo?orgId=1&refresh=500ms&from=now-5m&to=now](http://localhost:3000/d/qdb-iot-demo/device-data-questdb-demo?orgId=1&refresh=500ms&from=now-5m&to=now) . User is `admin` and password `quest`.


In [1]:
from questdb.ingress import Sender, IngressError, TimestampNanos
import os
import sys
import random
import time

HTTP_ENDPOINT = os.getenv('QUESTDB_HTTP_ENDPOINT', 'questdb:9000')

DEVICE_TYPES = ["blue", "red", "green", "yellow"]
ITER = 1000
BATCH = 100
DELAY = 1
MIN_LAT = 19.50139
MAX_LAT = 64.85694
MIN_LON = -161.75583
MAX_LON = -68.01197


def send(http_endpoint: str = HTTP_ENDPOINT, auth=None):
    try:
        conf = f'http::addr={http_endpoint};'
        with Sender.from_conf(conf) as sender:
            for it in range(ITER):
                for i in range(BATCH):
                    sender.row(
                        'iot_data',
                        symbols={'device_type': random.choice(DEVICE_TYPES)},
                        columns={
                                    'duration_ms': random.randint(0, 4000),
                                    "lat": random.uniform(MIN_LAT, MAX_LAT),
                                    "lon": random.uniform(MIN_LON, MAX_LON),
                                    "measure1": random.randint(-2147483648, 2147483647),
                                    "measure2": random.randint(-2147483648, 2147483647),
                                    "speed": random.randint(0, 100)
                        },
                        at=TimestampNanos.now())                
                sys.stdout.write(f'sent : {BATCH} rows\n')
                
                time.sleep(DELAY)
    except IngressError as e:
        sys.stderr.write(f'Got error: {e}')


if __name__ == '__main__':
    sys.stdout.write(f'Ingestion started. Connecting to {HTTP_ENDPOINT}\n')
    send()

Ingestion started. Connecting to host.docker.internal:9000
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows
sent : 100 rows


KeyboardInterrupt: 